In [ ]:
import pandas as pd
from neuralprophet import NeuralProphet, set_random_seed, set_log_level
from sklearn.metrics import mean_absolute_error
set_random_seed(0)
set_log_level('ERROR')

In [ ]:
# Đọc dữ liệu từ file CSV
df = pd.read_csv("VN30_data.csv")
print(df.dtypes)
# Xử lý tên cột (loại bỏ khoảng trắng thừa)
df.columns = df.columns.str.strip()

# Đảm bảo cột 'ds' là kiểu datetime
df['ds'] = pd.to_datetime(df['Date'], format='%d-%m-%y', errors='coerce')

df = df.drop(columns=['Date'])

# Kiểm tra các tên cột
print(df.columns)

# Loại bỏ các dòng có giá trị 'ds' trùng lặp
df = df.drop_duplicates(subset=['ds'], keep='first')

# Kiểm tra xem có bất kỳ giá trị thiếu nào trong cột 'ds'
if df['ds'].isnull().any():
    df = df.dropna(subset=['ds'])  # Loại bỏ dòng có giá trị thiếu trong 'ds'

df.reset_index(inplace=True)
df = df.drop(columns=['index'])

# Sắp xếp data
df = df.sort_values(by='ds')

In [ ]:
# Chọn các cột đặc trưng và cột mục tiêu
df_target = df[['ds', 'High', 'Low', 'Volume', 'Close']].copy()

# Đổi tên cột 'Close' thành 'y' để làm cột mục tiêu
df_target['y'] = df_target['Close']
df_target = df_target[['ds', 'y', 'High', 'Low', 'Volume']]  # Chỉ giữ lại cột 'ds' và cột 'y'

In [ ]:
# Hàm này để lấy dự đoán cho đúng ý mình, không cần chỉnh gì
def modify(df, forecast=5):
    # figure out where 'yhat1' lives
    col_idx = df.columns.get_loc('yhat1')

    # loop 1…9
    for i in range(1, forecast):
        # compute which yhatN we need
        which = forecast - i + 1          # 10→1, 9→2, …, 2→9
        val   = df.iloc[-i][f'yhat{which}'] 
        # row number = -i (so i=1 → last row, i=9 → 9th-from-last)
        df.iat[-i, col_idx] = val

In [ ]:
# Hàm đánh giá mô hình
def evaluate_model(params, df_target):
    model = NeuralProphet(
        n_lags=params['n_lags'],
        n_forecasts=params['n_forecasts'],
        learning_rate=params['learning_rate'],
        epochs=params['epochs'],
        n_changepoints=params['n_changepoints'],
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        trend_reg=0.5,                  
        trend_reg_threshold=True,
        normalize='standardize',
        drop_missing=True
    )
    
    # Thêm các cột đặc trưng như lagged regressors (nếu cần)
    for col in ['High', 'Low', 'Volume']:
        model.add_lagged_regressor(col)

    # Bỏ n_forecasts ngày cuối để test
    df_train = df_target[:-params['n_forecasts']]

    # Huấn luyện mô hình
    model.fit(df_train)
    
    # Dự báo
    future = model.make_future_dataframe(df_train, periods=params['n_forecasts'])
    forecast = model.predict(future)

    modify(forecast, params['n_forecasts'])
    
    # Tính toán lỗi MAE
    mae = mean_absolute_error(df_target[-params['n_forecasts']:]['y'], forecast['yhat1'].tail(params['n_forecasts']))
    return mae

In [ ]:
# Định nghĩa các giá trị tham số cần tìm kiếm
param_grid = {
    'n_lags': [4, 6, 8, 10],
    'n_forecasts': [1, 3],
    'learning_rate': [0.001, 0.01, 0.1],
    'epochs': [100, 200, 300],
    'n_changepoints': [3, 5, 7],
}

# Tìm kiếm thủ công qua các tham số
best_mae = float('inf')
best_params = None

for n_lags in param_grid['n_lags']:
    for n_forecasts in param_grid['n_forecasts']:
        for learning_rate in param_grid['learning_rate']:
            for epochs in param_grid['epochs']:
                for n_changepoints in param_grid['n_changepoints']:
                    params = {
                        'n_lags': n_lags,
                        'n_forecasts': n_forecasts,
                        'learning_rate': learning_rate,
                        'epochs': epochs,
                        'n_changepoints': n_changepoints
                    }
                    mae = evaluate_model(params, df_target)
                    print(f"Parameters: {params}, MAE: {mae}")
                    
                    if mae < best_mae:
                        best_mae = mae
                        best_params = params

print(f"Best parameters: {best_params} with MAE: {best_mae}")